In [15]:
import numpy as np
import cv2

In [16]:
# read video
cap = cv2.VideoCapture('../input/movie/raw/bacteria.mp4')

# algorithm
model_dict = { 
    'CNT': cv2.bgsegm.createBackgroundSubtractorCNT(), 
    'CMG': cv2.bgsegm.createBackgroundSubtractorGMG(),
    'GSOC': cv2.bgsegm.createBackgroundSubtractorGSOC(),
    'LSBP': cv2.bgsegm.createBackgroundSubtractorLSBP(),
    'MOG': cv2.bgsegm.createBackgroundSubtractorMOG(),
    'KNN': cv2.createBackgroundSubtractorKNN(),
    'MOG2': cv2.createBackgroundSubtractorMOG2()
}

# config
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter(
    '../output/movie/output_binary.mp4',
    fourcc,
    fps,
    (width, height),
    isColor = False
)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    model = model_dict['KNN']

    mask = model.apply(frame)

    # binarization using Otsu's method
    _, th = cv2.threshold(mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # configure the kernel
    kernel = np.ones((10,10),np.uint8)

    # morphological transformation(Dilation)
    th_dilation = cv2.dilate(th,kernel,iterations = 1)

    cv2.imshow('Mask', th_dilation)

    writer.write(th_dilation)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
writer.release()
cv2.destroyAllWindows()